# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

## <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks

/home/jdowling/anaconda3/envs/book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Getting the current date
today = datetime.date.today()
# start_day = today - datetime.timedelta(days = 100)
country="sweden"
city="stockholm"
street="stockholm-hornsgatan-108-gata"

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works:443/p/5240
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [4]:
# Retrieve the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=1,
)

## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [5]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=5,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

External IP not configured for the Istio ingress gateway, the Hopsworks client will be used for model inference instead
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='2.5548958E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float'], gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style="color:#ff5f27;">✨ Get Weather Forecast Features with Feature View   </span>



In [7]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

f = weather_fg.read()

Finished: Reading data from Hopsworks, using Hive (2.18s) 


In [8]:
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
a = air_quality_fg.read()
a = a.sort_values(by=['date'])
a

Finished: Reading data from Hopsworks, using Hive (2.23s) 


,date,pm25,country,city,street
1465,2017-10-04 02:00:00,13.0,sweden,stockholm,stockholm-hornsgatan-108-gata
1076,2017-10-05 02:00:00,9.0,sweden,stockholm,stockholm-hornsgatan-108-gata
1917,2017-10-06 02:00:00,8.0,sweden,stockholm,stockholm-hornsgatan-108-gata
1941,2017-10-07 02:00:00,13.0,sweden,stockholm,stockholm-hornsgatan-108-gata
1355,2017-10-08 02:00:00,8.0,sweden,stockholm,stockholm-hornsgatan-108-gata
...,...,...,...,...,...
1569,2024-02-29 01:00:00,38.0,sweden,stockholm,stockholm-hornsgatan-108-gata
1312,2024-03-01 01:00:00,46.0,sweden,stockholm,stockholm-hornsgatan-108-gata
2143,2024-03-02 01:00:00,59.0,sweden,stockholm,stockholm-hornsgatan-108-gata
2232,2024-03-03 01:00:00,48.0,sweden,stockholm,stockholm-hornsgatan-108-gata


In [9]:
batch_data = f[f['date'] >= str(today)]
batch_data = batch_data.sort_values(by=['date'])
batch_data

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
2346,2024-03-05 01:00:00,2.50,0.0,15.941944,64.592320,stockholm
2350,2024-03-06 01:00:00,4.15,0.0,6.725354,74.475830,stockholm
2348,2024-03-07 01:00:00,6.70,0.0,7.895416,294.227720,stockholm
2341,2024-03-08 01:00:00,3.20,0.0,11.609651,7.124930,stockholm
2343,2024-03-09 01:00:00,0.30,0.0,7.100310,59.534540,stockholm
2349,2024-03-10 01:00:00,0.80,0.0,15.281989,74.981590,stockholm
2342,2024-03-11 01:00:00,2.50,0.0,11.609651,82.875084,stockholm
2345,2024-03-12 01:00:00,5.55,0.0,8.049845,169.695200,stockholm
2351,2024-03-13 01:00:00,5.45,0.0,13.979871,168.111360,stockholm
2352,2024-03-14 01:00:00,3.85,0.2,15.496736,177.337050,stockholm


In [10]:
batch_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 2346 to 2354
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         11 non-null     datetime64[ns]
 1   temperature_2m_mean          11 non-null     float64       
 2   precipitation_sum            11 non-null     float64       
 3   wind_speed_10m_max           11 non-null     float64       
 4   wind_direction_10m_dominant  11 non-null     float64       
 5   city                         11 non-null     object        
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 616.0+ bytes


In [11]:
# batch_data = feature_view.get_batch_data(start_time=today, event_time=True, primary_key=True)
# pred_df = batch_data.drop(columns=['date'])
# print(feature_view.query.to_string())

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [12]:
batch_data['predicted_pm25'] = retrieved_xgboost_model.predict(
    batch_data[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']])
batch_data

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,predicted_pm25
2346,2024-03-05 01:00:00,2.50,0.0,15.941944,64.592320,stockholm,36.657509
2350,2024-03-06 01:00:00,4.15,0.0,6.725354,74.475830,stockholm,56.097218
2348,2024-03-07 01:00:00,6.70,0.0,7.895416,294.227720,stockholm,45.975529
2341,2024-03-08 01:00:00,3.20,0.0,11.609651,7.124930,stockholm,30.115486
2343,2024-03-09 01:00:00,0.30,0.0,7.100310,59.534540,stockholm,43.068813
2349,2024-03-10 01:00:00,0.80,0.0,15.281989,74.981590,stockholm,31.776596
2342,2024-03-11 01:00:00,2.50,0.0,11.609651,82.875084,stockholm,37.584156
2345,2024-03-12 01:00:00,5.55,0.0,8.049845,169.695200,stockholm,42.625175
2351,2024-03-13 01:00:00,5.45,0.0,13.979871,168.111360,stockholm,30.744324
2352,2024-03-14 01:00:00,3.85,0.2,15.496736,177.337050,stockholm,28.910917


In [13]:
batch_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 2346 to 2354
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         11 non-null     datetime64[ns]
 1   temperature_2m_mean          11 non-null     float64       
 2   precipitation_sum            11 non-null     float64       
 3   wind_speed_10m_max           11 non-null     float64       
 4   wind_direction_10m_dominant  11 non-null     float64       
 5   city                         11 non-null     object        
 6   predicted_pm25               11 non-null     float32       
dtypes: datetime64[ns](1), float32(1), float64(4), object(1)
memory usage: 660.0+ bytes


In [14]:
batch_data['street'] = street
batch_data['city'] = city
batch_data['country'] = country
# Fill in the number of days before the date on which you made the forecast (base_date)
batch_data['days_before_forecast_day'] = range(1, len(batch_data)+1)

In [26]:
batch_data[['date', 'predicted_pm25']]

,date,predicted_pm25
2346,2024-03-05 01:00:00,36.657509
2350,2024-03-06 01:00:00,56.097218
2348,2024-03-07 01:00:00,45.975529
2341,2024-03-08 01:00:00,30.115486
2343,2024-03-09 01:00:00,43.068813
2349,2024-03-10 01:00:00,31.776596
2342,2024-03-11 01:00:00,37.584156
2345,2024-03-12 01:00:00,42.625175
2351,2024-03-13 01:00:00,30.744324
2352,2024-03-14 01:00:00,28.910917


### Create Forecast Graph
Draw a graph of the predictions with dates as a PNG and save it to the github repo
Show it on github pages

In [27]:
import plotly.express as px

fig = px.line(batch_data, x="date", y=['predicted_pm25'])
filename = "../../docs/air-quality/assets/img/pm25_forecast.png"
fig.write_image(filename)

In [15]:
# Get or create feature group
monitor_fg = fs.get_or_create_feature_group(
    name='aq_monitoring',
    description='Air Quality prediction monitoring',
    version=1,
    primary_key=['country','street','date', 'days_before_forecast_day'],
    event_time="date"
)

In [16]:
monitor_fg.insert(batch_data, wait=True)

Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████████████████████████| Rows 11/11 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: aq_monitoring_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/5240/jobs/named/aq_monitoring_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f2f49f11930>, None)

In [17]:
from hsfs.feature import Feature

# We will create a hindcast chart for  only the forecasts made 1 day beforehand
monitoring_df = monitor_fg.filter(Feature("days_before_forecast_day") == 1).read()
monitoring_df

Finished: Reading data from Hopsworks, using Hive (1.85s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,predicted_pm25,street,country,days_before_forecast_day
0,2024-03-05 02:00:00,2.5,0.0,15.941944,64.59232,stockholm,36.65751,stockholm-hornsgatan-108-gata,sweden,1


In [18]:
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
air_quality_df = air_quality_fg.read()
air_quality_df

Finished: Reading data from Hopsworks, using Hive (2.25s) 


,date,pm25,country,city,street
0,2017-10-18 02:00:00,10.0,sweden,stockholm,stockholm-hornsgatan-108-gata
1,2020-06-17 02:00:00,30.0,sweden,stockholm,stockholm-hornsgatan-108-gata
2,2023-04-12 02:00:00,62.0,sweden,stockholm,stockholm-hornsgatan-108-gata
3,2020-03-22 01:00:00,16.0,sweden,stockholm,stockholm-hornsgatan-108-gata
4,2018-11-11 01:00:00,57.0,sweden,stockholm,stockholm-hornsgatan-108-gata
...,...,...,...,...,...
2265,2017-10-12 02:00:00,10.0,sweden,stockholm,stockholm-hornsgatan-108-gata
2266,2020-10-08 02:00:00,17.0,sweden,stockholm,stockholm-hornsgatan-108-gata
2267,2018-04-23 02:00:00,17.0,sweden,stockholm,stockholm-hornsgatan-108-gata
2268,2019-03-27 01:00:00,33.0,sweden,stockholm,stockholm-hornsgatan-108-gata


In [19]:
air_quality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     2270 non-null   datetime64[ns]
 1   pm25     2270 non-null   float64       
 2   country  2270 non-null   object        
 3   city     2270 non-null   object        
 4   street   2270 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 88.8+ KB


In [20]:
outcome_df = air_quality_df[['date', 'pm25']]
preds_df =  monitoring_df[['date', 'predicted_pm25']]

hindcast_df = pd.merge(preds_df, outcome_df, on="date")
hindcast_df = hindcast_df.sort_values(by=['date'])
hindcast_df

,date,predicted_pm25,pm25


In [22]:
import plotly.express as px

fig = px.line(hindcast_df, x="date", y=['pm25', 'predicted_pm25'])
filename = "../../docs/air-quality/assets/img/pm25_hindcast_1day.png"
fig.write_image(filename)